In [ ]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import mpl_toolkits
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import windrose
import matplotlib.ticker as ticker
import math

import ee
import geemap

# import the cartoee functionality from geemap
from geemap import cartoee

geemap.ee_initialize()

In [ ]:
# Adding an interactive map.
Map = geemap.Map(basemap='ROADMAP', center=[24, 91], zoom=7, height=600, draw = True)

# Adding the TERRAIN basemaps.
Map.add_basemap("TERRAIN")
Map

In [ ]:
# Taking a draw feature from the map.
feature = Map.draw_last_feature

# If drawing does not exist then taking a polygon as the default version.
if feature is None:
    geom = ee.Geometry.Polygon([[[91.781, 21.97],
                                 [91.781, 21.70],
                                 [91.93, 21.70],
                                 [91.93, 21.97]]])
    feature = ee.Feature(geom, {})

# Define the AOI.
AOI = feature.geometry()

# Visualize the map.
Map.addLayer(AOI, {}, "AOI")
Map

In [ ]:
# Use Landsat 8 surface reflectance data.
Landsat_image_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            # filtering image collection by AOI
            .filterBounds(AOI)
            # filtering image collection by Dates
            .filterDate('2023-01-01', '2023-12-31')
            # filtering image collection by Metadata (Metadata: Cloud cover)
           .filter(ee.Filter.lt('CLOUD_COVER',50))
           .sort('CLOUD_COVER',False))

print ('Number of landsat images in AOI: ', Landsat_image_collection.size().getInfo())

In [ ]:
# Apply scaling factors.
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Function for cloud masking
def fmask(image):
    # QA_PIXEL = Pixel quality attributes generated from the CFMASK algorithm.
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask)

# Applying scale factor 
with_cloud_image = Landsat_image_collection.map(apply_scale_factors)

# Cloud Masked Image
cloud_masked_image = Landsat_image_collection.map(fmask)

# Getting all image collection for AOI and calculate median and clip the final image.
median_image = cloud_masked_image.median().clip(AOI)


# Specifying visualization parameters.
vis_natural = {
    'bands': ['SR_B7', 'SR_B6', 'SR_B4'],
    'min': 0.0,
    'max': 0.3,
}

# Adding image to the map.
Map_all = geemap.Map(height=500)
Map_all.addLayer(median_image, vis_natural, 'Landsat-8 composite')
Map_all.centerObject(median_image, 11)
Map_all

In [ ]:
wind_data_file_path = r"E:\Script\BMD_Wind_Data_Processing\New folder\Output\Kutubdia_3H_Wind.csv"

# Reading the csv file and removing the non-number data 
# from the wind speed and wind direction data column
df = pd.read_csv(wind_data_file_path, low_memory=False)
col_name  = ['Wind Speed (knots)', 'Wind Direction (degree)']
df[col_name] = df[col_name].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df.dropna(subset=['Wind Speed (knots)', 'Wind Direction (degree)'], inplace = True)

# Setting the wind speed and wind direction column and plotting the windrose
ws = df['Wind Speed (knots)'].astype(float)
wd = df['Wind Direction (degree)'].astype(float) * 10

 # Setting the Number of wind class and step (needs to be adjusted)
bins = np.arange(0.01,10,2)


minlon, maxlon, minlat, maxlat = (91.70, 91.92, 21.70, 21.94)

font={'family':'Times New Roman',
             'weight':'normal',
             'size':12}

plt.rc('font',**font)

proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(6, 12))

# First subplot (left)
img_ax = plt.subplot(121, projection=ccrs.PlateCarree())
img_ax.set_extent([minlon, maxlon, minlat, maxlat], crs=proj)

# Second subplot (right)
ax = WindroseAxes(fig, [1.10, 0.2, 0.7, 0.7])  # Adjust position as needed [left, bottom, width, height]
fig.add_axes(ax)


img_ax = cartoee.get_map(median_image, vis_params=vis_natural, basemap="ROADMAP", zoom_level=8)

# Coordinates of the station we were measuring windspeed
passy_lon, passy_lat = (91.85, 21.82)

xticks = np.arange(91.80, 91.93, step = 0.02)
img_ax.set_xticks(xticks)
img_ax.tick_params(axis='x', labelbottom= False, labelsize=12, pad = 3)
# img_ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
img_ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.2f}\u00b0E"))

yticks = np.arange(21.70, 21.95, step = 0.04)
img_ax.set_yticks(yticks)
img_ax.tick_params(axis='y', labelleft= False, labelrotation=90, labelsize=12, pad = 3)
# img_ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
img_ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.2f}\u00b0N"))

plt.setp(img_ax.get_yticklabels(), va='center')
plt.setp(img_ax.get_xticklabels(), ha='center')

# Inset axe with size relative to main axe
height_deg = 0.03
wrax_passy = inset_axes(img_ax,
        width="100%",                        # size in % of bbox
        height="100%",                       # size in % of bbox
        #loc='center',  # don't know why, but this doesn't work.
        # specify the center lon and lat of the plot, and size in degree
        bbox_to_anchor=(passy_lon-height_deg/2, passy_lat-height_deg/2, height_deg, height_deg),
        bbox_transform=img_ax.transData,
        axes_class=windrose.WindroseAxes,
        )

wrax_passy.bar(wd, ws, normed=True, nsector=16, bins=bins, opening=0.8, edgecolor='k', cmap =cm.jet,
               blowto = None)

for loc_ax in [wrax_passy]:
    loc_ax.tick_params(labelleft=False, labelbottom=False)


# PLOTTING WIND ROSE

ax.bar(wd, ws, normed=True, nsector=16, bins=bins, opening=0.8, edgecolor='k', cmap =cm.jet,
      blowto = None)

# Direction (x-axis) LABEL FORMATTING

xtick_labels = ax.get_xticklabels()
formatted_xticks_label = ['{y}\n({x:.0f}\u00b0)'.format( y = xlabel._text, x=math.degrees(xlabel._x)) \
                          for xlabel in xtick_labels]
ax.tick_params(axis='x', labelsize=12, pad=12)
ax.set_xticklabels(formatted_xticks_label)


# Wind Speed (y-axis) LABEL FORMATTING

diff = ax.get_yticklabels()[1]._y - ax.get_yticklabels()[0]._y
step_value = round(diff*2)/2
max_value = round(ax.get_yticklabels()[-1]._y) + step_value

np.arange(0.0, max_value,step_value)
# ax.set_ylim(0,21)
ax.set_yticks(np.arange(0.0, max_value, step = step_value))
ax.set_yticklabels(np.arange(0.0, max_value, step = step_value))


# LEGEND FORMATTING
legend = ax.set_legend(title = 'Wind Speed (knots)', title_fontproperties={'weight':'bold', 'size':14},
                       loc='upper right', prop = {'size':14}, frameon=True, bbox_to_anchor=(0.70, -0.15))
legend._legend_box.align = "left"

legend_label_list = []

for label_txt in ax.get_legend().get_texts():

    if label_txt ==  ax.get_legend().get_texts()[-1]:
        legend_label_list.append('{} {}'.format(label_txt.get_text()[0], label_txt.get_text()[1:]))

    else:
        new_label = '{}-{}'.format(label_txt.get_text().split(':')[0][1:], label_txt.get_text().split(':')[-1][:-1])
        legend_label_list.append(new_label)


legend_texts = legend.get_texts()

for i,label in enumerate(legend_label_list):
    legend_texts[i].set_text(label)

plt.setp(legend.get_texts(), fontsize=14)

fig.suptitle('Windrose Plot for Kutubdia', fontsize=22, x=0.88, y = 0.935)

# Adjusting layout to prevent overlap
plt.tight_layout()

plt.savefig(r'test.jpg', dpi=300, bbox_inches = 'tight')
    
plt.show()